In [1]:
import multiprocessing
from GCForest import gcForest
import pandas as pd
import numpy as np
import pickle 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import Imputer
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()

In [2]:
#读取Y
all_df_y = pickle.load(open("./dump_file/all_df_y","rb"))
all_df_y['2cArousal'] = 0
all_df_y['2cArousal'][all_df_y['valence'] >= 5] = 1
all_df_y['2cValence'] = 0
all_df_y['2cValence'][all_df_y['valence'] >= 5] = 1
print(all_df_y.head(5))

y = all_df_y[['2cValence']]
#y = all_df_y[["2cArousal"]]
for seed in [0,100,200,300,400,500,600,700,800,900]:
    print("[seed:{}]****************************************************".format(seed))
    xTrainIdx = pickle.load(open("./dump_file/xTrainIdx","rb"))
    xTestIdx = pickle.load(open("./dump_file/xTestIdx","rb"))
    y_tr = y.loc[xTrainIdx]
    y_te = y.loc[xTestIdx]

    myWindowsSize = 256
    myStrideSize = 64
    gcf = gcForest(shape_1X=8064, window=myWindowsSize, stride=myStrideSize,tolerance=0.0,n_cascadeRF=1, 
                   min_samples_mgs=0.2, min_samples_cascade=0.2,cascade_layer = 5,cascade_test_size = 0.1
                   ,n_jobs=19)

    #这部分是时域的
    for eegCH in range(1,33,1):
        filePath = "./dump_file/CH{}mgsTrainVector_{}_{}_{}".format(eegCH,myWindowsSize,myStrideSize,seed)
        locals()['CH{}mgsTrainVector'.format(eegCH)] = pickle.load(open(filePath,"rb"))
        filePath = "./dump_file/CH{}mgsTestVector_{}_{}_{}".format(eegCH,myWindowsSize,myStrideSize,seed)
        locals()['CH{}mgsTestVector'.format(eegCH)]= pickle.load(open(filePath,"rb"))
    X_tr_vector = locals()['CH{}mgsTrainVector'.format(1)]
    X_te_vector = locals()['CH{}mgsTestVector'.format(1)]
    #拼接MGS处理的各个通道向量---》cascade forest
    for eegCH in range(2,33,1):
        nextTrainVector = locals()['CH{}mgsTrainVector'.format(eegCH)]
        nextTestVector = locals()['CH{}mgsTestVector'.format(eegCH)]
        X_tr_vector = np.concatenate([X_tr_vector,nextTrainVector],axis=1)
        X_te_vector = np.concatenate([X_te_vector,nextTestVector],axis=1)
    print(X_tr_vector.shape)
    print(X_te_vector.shape)

    #这部分是频域的
    if False:
        for eegCH in range(1,33,1):
            filePath = "./dump_file_V2/CH{}mgsTrainVector_{}_{}_{}".format(eegCH,myWindowsSize,myStrideSize,seed)
            locals()['CH{}mgsTrainVector'.format(eegCH)] = pickle.load(open(filePath,"rb"))
            filePath = "./dump_file_V2/CH{}mgsTestVector_{}_{}_{}".format(eegCH,myWindowsSize,myStrideSize,seed)
            locals()['CH{}mgsTestVector'.format(eegCH)]= pickle.load(open(filePath,"rb"))\
        #在已经有时域数据的基础上拼接MGS处理的各个通道向量---》cascade forest
        for eegCH in range(1,33,1):
            nextTrainVector = locals()['CH{}mgsTrainVector'.format(eegCH)]
            nextTestVector = locals()['CH{}mgsTestVector'.format(eegCH)]
            X_tr_vector = np.concatenate([X_tr_vector,nextTrainVector],axis=1)
            X_te_vector = np.concatenate([X_te_vector,nextTestVector],axis=1)
        print(X_tr_vector.shape)
        print(X_te_vector.shape)

    #这部分是BVP
    filePath = "./dump_file_V2/BVP_mgsTrainVector_{}_{}_{}".format(myWindowsSize,myStrideSize,seed)
    BVP_mgsTrainVector = pickle.load(open(filePath,"rb"))
    filePath = "./dump_file_V2/BVP_mgsTestVector_{}_{}_{}".format(myWindowsSize,myStrideSize,seed)
    BVP_mgsTestVector = pickle.load(open(filePath,"rb"))
    X_tr_vector = np.concatenate([X_tr_vector,BVP_mgsTrainVector],axis=1)
    X_te_vector = np.concatenate([X_te_vector,BVP_mgsTestVector],axis=1)
    print(X_tr_vector.shape)
    print(X_te_vector.shape)
    #这部分是RSP
    filePath = "./dump_file_V2/RSP_mgsTrainVector_{}_{}_{}".format(myWindowsSize,myStrideSize,seed)
    RSP_mgsTrainVector = pickle.load(open(filePath,"rb"))
    filePath = "./dump_file_V2/RSP_mgsTestVector_{}_{}_{}".format(myWindowsSize,myStrideSize,seed)
    RSP_mgsTestVector = pickle.load(open(filePath,"rb"))
    X_tr_vector = np.concatenate([X_tr_vector,RSP_mgsTrainVector],axis=1)
    X_te_vector = np.concatenate([X_te_vector,RSP_mgsTestVector],axis=1)
    print(X_tr_vector.shape)
    print(X_te_vector.shape)
    #这部分是GSR
    filePath = "./dump_file_V2/GSR_mgsTrainVector_{}_{}_{}".format(myWindowsSize,myStrideSize,seed)
    RSP_mgsTrainVector = pickle.load(open(filePath,"rb"))
    filePath = "./dump_file_V2/GSR_mgsTestVector_{}_{}_{}".format(myWindowsSize,myStrideSize,seed)
    RSP_mgsTestVector = pickle.load(open(filePath,"rb"))
    X_tr_vector = np.concatenate([X_tr_vector,RSP_mgsTrainVector],axis=1)
    X_te_vector = np.concatenate([X_te_vector,RSP_mgsTestVector],axis=1)
    print(X_tr_vector.shape)
    print(X_te_vector.shape)
    #如有缺失值，填充下
    X_tr_vector_fillna= pd.DataFrame(X_tr_vector).fillna(0).values
    X_te_vector_fillna= pd.DataFrame(X_te_vector).fillna(0).values
    _ = gcf.cascade_forest(X_tr_vector_fillna, y_tr)
    pred_proba = gcf.cascade_forest(X_te_vector_fillna)
    tmp = np.mean(pred_proba, axis=0)
    preds = np.argmax(tmp, axis=1)
    print("ACC",accuracy_score(y_true=y_te, y_pred=preds))
    print("F1",f1_score(y_true=y_te, y_pred=preds))
    print("Recal",recall_score(y_true=y_te, y_pred=preds))
    print("Precision",precision_score(y_true=y_te, y_pred=preds))

       valence  arousal  dominance  liking  2cArousal  2cValence
s01_0     7.71     7.60       6.90    7.83          1          1
s01_1     8.10     7.31       7.28    8.47          1          1
s01_2     8.58     7.54       9.00    7.08          1          1
s01_3     4.94     6.01       6.12    8.06          0          0
s01_4     6.96     3.92       7.19    6.05          1          1
[seed:0]****************************************************
(896, 15744)
(384, 15744)
(896, 16236)
(384, 16236)
(896, 16728)
(384, 16728)
(896, 17220)
(384, 17220)
Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.6666666666666666
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.6666666666666666
ACC 0.671875
F1 0.7613636363636365
Recal 0.9262672811059908
Precision 0.6463022508038585
[seed:100]****************************************************
(896, 15744)
(384, 15744)
(896, 16236)
(384, 16236)
(896, 16728)
(384, 16728)
(896, 17220)
(384, 17220)
Adding/Training Layer, n_laye